[View in Colaboratory](https://colab.research.google.com/github/Bala1718/Emojyourself/blob/master/AD_Model.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty
import os
os.chdir('drive/Alzheimer_Dataset')


In [3]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, MaxPooling2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from sklearn.model_selection import train_test_split
import time
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import PIL

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import Callback

Using TensorFlow backend.


In [4]:
train_data_dir = 'dataset/train/'
validation_data_dir = 'dataset/test/'

In [5]:
batch_size = 32
#img_height = 160
#img_width = 160

img_height = 224
img_width = 224

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_batches = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        classes=['VeryMildDemented','NonDemented','ModerateDemented','MildDemented'],
        class_mode='categorical')

validation_batches = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        classes=['VeryMildDemented','NonDemented','ModerateDemented','MildDemented'],
        class_mode='categorical')

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [6]:
vgg16_model = keras.applications.vgg16.VGG16()

553476096/553467096 [==============================] - 19s 0us/step


In [7]:
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

model.layers.pop()

In [8]:
for layer in model.layers:
    layer.trainable = False

model.add(Dense(4,activation='softmax'))

In [28]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [28]:
#model.compile((Adam(lr=0.01),loss='categorical_crossentropy', metrics=['accuracy'])


In [0]:
st = time.time()
#model.fit_generator(train_batches,steps_per_epoch=80,validation_data=validation_batches,validation_steps=20,epochs=100,verbose=2)

model.fit_generator(train_batches,validation_data=validation_batches,epochs=100,verbose=2)
en = time.time()

Epoch 1/100


In [0]:
model.save('al_trial_model.h5')
model.save_weights('epoch1.h5')


In [0]:
predictions = model.predict_generator(test_batches, steps=10, verbose=0)

In [0]:
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))

In [0]:
cm_plots_labels = ['VeryMildDemented','NonDemented','ModerateDemented','MildDemented']
plot_confusion_matrix(cm, cm_plots_labels, title='Confusion Matrix')